In [12]:
import requests
import json
import csv
from datetime import datetime

def download_data(url):
    response = requests.get(url)  # Send a GET request to the URL
    data = response.json()  # Parse the response as JSON
    return data

def process_data(data):
    processed_data = []
    for item in data:
        # Extract the required attributes from each item in the data
        meteorite_id = item['id']
        name = item['name']
        name_type = item['nametype']
        recclass = item['recclass']
        mass = float(item['mass (g)']) if 'mass (g)' in item else None
        year = parse_year(item.get('year'))
        reclat = float(item['reclat']) if 'reclat' in item else None
        reclong = float(item['reclong']) if 'reclong' in item else None
        
        # Check if the 'year' key is present in the item dictionary
        if 'year' in item:
            year = parse_year(item['year'])
        else:
            year = None
        
        # Create a dictionary with the processed attributes
        processed_item = {
            'Meteorite ID': meteorite_id,
            'Name': name,
            'Name Type': name_type,
            'Class': recclass,
            'Mass (g)': mass,
            'Year': year,
            'Latitude': reclat,
            'Longitude': reclong,
            'Coordinates': [reclat, reclong]
        }
        processed_data.append(processed_item)
    
    return processed_data



def parse_year(year_string):
    if year_string is None:
        return None
    try:
        return datetime.strptime(year_string, '%Y-%m-%dT%H:%M:%S.%f')
    except ValueError:
        return None


def save_as_csv(data, filename):
    fieldnames = data[0].keys()
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# URL to download the data from
url = 'https://data.nasa.gov/resource/y77d-th95.json'

# Download the data
data = download_data(url)

# Process the data
processed_data = process_data(data)

# Save the processed data as a CSV file
save_as_csv(processed_data, 'meteorite_data.csv')

